In [2]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [4]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [7]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [11]:
trainer.train()

                                                 
 33%|███▎      | 125/375 [02:52<04:24,  1.06s/it]

{'eval_loss': 1.1317740678787231, 'eval_accuracy': 0.507, 'eval_runtime': 38.9609, 'eval_samples_per_second': 25.667, 'eval_steps_per_second': 3.208, 'epoch': 1.0}


                                                 
 67%|██████▋   | 250/375 [05:41<02:11,  1.05s/it]

{'eval_loss': 1.0585907697677612, 'eval_accuracy': 0.548, 'eval_runtime': 37.7615, 'eval_samples_per_second': 26.482, 'eval_steps_per_second': 3.31, 'epoch': 2.0}


                                                 
100%|██████████| 375/375 [08:31<00:00,  1.36s/it]

{'eval_loss': 1.0630725622177124, 'eval_accuracy': 0.569, 'eval_runtime': 37.5678, 'eval_samples_per_second': 26.619, 'eval_steps_per_second': 3.327, 'epoch': 3.0}
{'train_runtime': 511.797, 'train_samples_per_second': 5.862, 'train_steps_per_second': 0.733, 'train_loss': 1.0429724934895834, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=1.0429724934895834, metrics={'train_runtime': 511.797, 'train_samples_per_second': 5.862, 'train_steps_per_second': 0.733, 'total_flos': 789354427392000.0, 'train_loss': 1.0429724934895834, 'epoch': 3.0})

In [12]:
#trainer.save_model("path/to/model")